## Access the Downloaded Data

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from collections import Counter

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')

from astropy.table import Table
from astropy.cosmology import WMAP9 as cosmo
from astropy import units

from shared_astro_utils import matching_utils  # personal PyPI package

import shared_notebook_utils  # local script, may move to PyPI

ModuleNotFoundError: No module named 'shared_notebook_utils'

In [ ]:
def remove_bad_values(x):
    return x[(~np.isnan(x)) & (np.isfinite(x))]

In [ ]:
data_loc = 'catalogs/nsa_v1_0_1_ossy_ukidss_dr9_las_allwise.fits'

Including AllWISE at 5 arcseconds, we have 182128 rows. This is down only a few hundred from 182956.

In [ ]:
data = Table.read(data_loc)

In [ ]:
pd.Series(data['PROGRAMNAME']).value_counts()

In [ ]:
data['PROGRAMNAME'] = list(map(lambda x: x.strip(), data['PROGRAMNAME']))
data = data[data['PROGRAMNAME'] == 'legacy']

In [ ]:
assert len(data) == 180318

#### Redshifts (from NSA)

In [ ]:
n_bins = 50

In [ ]:
plt.hist(data['Z'], bins=n_bins)
plt.xlabel('z')
plt.ylabel('Galaxy Count')

We might expect a peak from the balance between completeness and volume probed. This does seem kinda sharp nonetheless.

## SDSS Photometry

#### Absolute Magnitudes

In [ ]:
ordered_sdss_bands = ['F', 'N', 'U', 'G', 'R', 'I', 'Z']
sdss_mags = list(map(lambda x: x + '_MAG_ABSOLUTE', ordered_sdss_bands))

for n, abs_mag_name in enumerate(sdss_mags):
    data[abs_mag_name] = data['ELPETRO_ABSMAG'][:, n]


In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(8, 3))

g_band_mags = np.array(data['G_MAG_ABSOLUTE'])
g_band_mags = remove_bad_values(g_band_mags)
g_band_mags = g_band_mags[(g_band_mags < -14) & (g_band_mags > -23)]  # remove crazy tails

r_band_mags = np.array(data['R_MAG_ABSOLUTE'])
r_band_mags = remove_bad_values(r_band_mags)
r_band_mags = r_band_mags[(r_band_mags < -14) & (r_band_mags > -23)]  # remove crazy tails

axes[0].set_ylabel('Galaxies')

axes[0].hist(g_band_mags, bins=n_bins)
axes[0].set_xlabel('SDSS g absolute mag')


axes[1].hist(r_band_mags, bins=n_bins)
axes[1].set_xlabel('SDSS r absolute mag')

fig.tight_layout()

#### Apparent Magnitudes (from NSA)

This is currently estimated from the NSA estimated absolute magnitudes - but this is obviously backwards. It would be better to get these direct from SDSS observations - they are surely recorded somewhere.

Chris says the apparent magnitudes can be grabbed from a hidden casjobs table called 'galaxies'. I should go get these instead of working 'backwards' with cosmology.

In [ ]:
data['DISTANCE_MODULUS'] = cosmo.distmod(data['Z'])
data['DISTANCE_MODULUS_LIST'] = np.array(list(map(lambda x: [x for n in range(7)], data['DISTANCE_MODULUS'])))
data['ELPETRO_APPARENT_MAG'] = data['ELPETRO_ABSMAG'] + data['DISTANCE_MODULUS_LIST']

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(8, 3))

g_band_mags = np.array(data['ELPETRO_APPARENT_MAG'])[:, 3] # g is NSA 4th value: FNugriz
g_band_mags = remove_bad_values(g_band_mags)
g_band_mags = g_band_mags[(g_band_mags > 14) & (g_band_mags < 20)]  # remove crazy tails

r_band_mags = np.array(data['ELPETRO_APPARENT_MAG'])[:, 4] # r is NSA 5th value: FNugriz
r_band_mags = remove_bad_values(r_band_mags)
r_band_mags = r_band_mags[(r_band_mags > 14) & (r_band_mags < 20)]  # remove crazy tails

axes[0].set_ylabel('Galaxies')

axes[0].hist(g_band_mags, bins=n_bins)
axes[0].set_xlabel('SDSS g apparent mag')


axes[1].hist(r_band_mags, bins=n_bins)
axes[1].set_xlabel('SDSS r apparent mag')
axes[1].axvline(17.77, c='r')

fig.tight_layout()

The sudden drop is (we think) due to the SDSS selection cut. It's slightly deeper than r = 17.77 because leftover fibres were used to image fainter galaxies (according to Chris). The simplest thing to do would be to artifically impose the r = 17.77 cut. It would be better to include the fainter galaxies, but I'm not sure where to look for the selection function.

I still need to see how this matches up to GZ morphology selections. If the fainter ones would be excluded anyway, it's not useful.

#### SDSS Flux

#### Which flux values to use?

The NSA v1_0_1 (schema [here](https://data.sdss.org/datamodel/files/ATLAS_DATA/ATLAS_MAJOR_VERSION/nsa.html)) contains various flux measurements with unclear descriptions.


**Flux are quotes as raw (_FLUX), extinction-corrected (_NMGY), and extinction + kcorrected (_RNMGY)**

> SERSIC_FLUX: 	Two-dimensional, single-component Sersic fit flux in GALEX/SDSS FNugriz

This is the apparent (I think) measured flux.

> SERSIC_NMGY: Galactic-extinction corrected AB flux in GALEX/SDSS FNugriz used for K-correction (from SERSIC_FLUX)

This is the apparent AB (but AB is a mag system??) flux, corrected for galactic extinction.

> SERSIC_RNMGY: Reconstructed AB nanomaggies in GALEX/SDSS FNugriz from K-correction fit, for Sersic fluxes

This is the above flux, shifted between bands according to a small k-correction. This is clearer from the IDL documentation for the [k-correction routine used](http://cosmo.nyu.edu/mb144/kcorrect/kcorrect_help.html#K_RECONSTRUCT_MAGGIES).


**The same values are available with an elliptical aperture, rather than a sersic fit**

> ELPETRO_FLUX: Elliptical SDSS-style Petrosian flux in GALEX/SDSS FNugriz (using r-band aperture) 

> ELPETRO_NMGY: Galactic-extinction corrected AB flux in GALEX/SDSS FNugriz used for K-correction (from PETRO_FLUX) 

*(This is probably a typo - I think it would be from ELPETRO_FLUX)*

> ELPETRO_RNMGY: Reconstructed AB nanomaggies in GALEX/SDSS FNugriz from K-correction fit, for elliptical petrosian fluxes

**A few other fluxes are available, but only in raw form**

> PETRO_FLUX: Azimuthally-averaged SDSS-style Petrosian flux in FNugriz (GALEX-SDSS photometric systems) 

> FIBER_FLUX: 	Flux in 3-arcsec diameter aperture (not apodized) in FNugriz 


The [online docs](https://www.sdss.org/dr13/algorithms/magnitudes/) suggest petrosian (elliptical) flux for my usecase.

> Galaxies bright enough to be included in our spectroscopic sample have relatively high signal-to-noise ratio measurements of their Petrosian magnitudes. Since these magnitudes are model-independent and yield a large fraction of the total flux, roughly constant with redshift, petroMag is the measurement of choice for such objects.

I will use ELPETRO_RNMGY and hope that the k-correction is still better with than without for AGN SEDs.

In [ ]:
# repeat unpacking for apparent flux
sdss_apparent_fluxes = list(map(lambda x: x + '_FLUX_APPARENT', ordered_sdss_bands))

for n, abs_flux_name in enumerate(sdss_apparent_fluxes):
    data[abs_flux_name] = data['ELPETRO_RNMGY'][:, n] / (3.631e-6) * units.Jy # convert nanomaggie apparent flux to Jy

In [ ]:
data[[band + '_FLUX_APPARENT' for band in ordered_sdss_bands]][:5]

There's no such thing as 'absolute flux'. I would have thought you could use luminosity - flux in the complete sphere - calculated from the distance. But I don't see papers doing this?

Instead, let's make up 'absolute flux' in analogy with absolute magnitude - the flux at a reference distance of 10pc. This is convenient because we already have absolute magnitudes, so we can use a standard mag->flux calculation to convert to 'absolute' flux.

In [ ]:
for band in ordered_sdss_bands:  # SDSS uses (almost) an AB system. TODO: correct for zero point misalignment
    data[band + '_FLUX_ABSOLUTE'] = shared_notebook_utils.ab_mag_to_flux(data[band + '_MAG_ABSOLUTE'])

In [ ]:
fig, ax = plt.subplots()
bins=50
pd.Series(np.log10(data['U_FLUX_APPARENT'])).hist(ax=ax, bins=bins, alpha=0.5, log=True)
pd.Series(np.log10(data['U_FLUX_ABSOLUTE'])).hist(ax=ax, bins=bins, alpha=0.5, log=True)
ax.legend(['Apparent U Flux Density', 'Absolute U Flux Density'])
ax.set_xlabel('Log Flux (Jy)')
ax.set_ylabel('Galaxies')

#### UKIDSS Apparent Magnitudes

In [ ]:
ukidss_cols = [
    'JName',
    'ra_xmatch',
    'dec_xmatch',
    'yAperMag3',
    'j_1AperMag3',
    'hAperMag3',
    'kAperMag3',
    'yAperMag3Err',
    'j_1AperMag3Err',
    'hAperMag3Err',
    'kAperMag3Err',
    'sourceID',
    'mode',
    'epoch',
    'mergedClass',
    'angDist',
    'SDSS_ID'
]


apparent_mag_bands = [
    'yAperMag3',
    'j_1AperMag3',
    'hAperMag3',
    'kAperMag3',
]

fig, axes = plt.subplots(ncols=len(apparent_mag_bands), figsize=(20, 3))
for axes_n, mag in enumerate(apparent_mag_bands):
    axes[axes_n].hist(remove_bad_values(data[mag]), bins=50)
    axes[axes_n].set_xlabel(mag)
axes[0].set_ylabel('Galaxies')
fig.tight_layout()

"Y=20.2, J=19.6, H=18.8, K=18.2" is indeed the deepest that UKIDSS has any objects.

The data release papers claim this is the limit for a 5 sigma detection in 2 arcsec aperture. But surely galaxies only get more common at fainter depths - as seen with SDSS. Why the gradual tail-off?

The answer might be that the SDSS spectroscopy cut is brighter than the UKIDSS depth limit, causing SDSS to have a sharp cut-off and UKIDSS to have a gradual decline (as faint galaxies become missed by SDSS, even though they are imaged by UKIDSS).

If so, and we plotted UKIDSS *without* cross-matching, we would expect to see a continual increase to the limiting depth and then a sharp cut-off. 

**TODO: I should check this.**


#### UKIDSS Absolute Magnitudes

Here, it actually makes sense to use cosmology to convert apparent magnitudes to (approximate i.e. without k-correction) absolute magnitues

In [ ]:
# Note: these are not k-corrected
data['Y_MAG_ABSOLUTE'] = data['yAperMag3'] - data['DISTANCE_MODULUS']
data['J_MAG_ABSOLUTE'] = data['j_1AperMag3'] - data['DISTANCE_MODULUS']
data['H_MAG_ABSOLUTE'] = data['hAperMag3'] - data['DISTANCE_MODULUS']
data['K_MAG_ABSOLUTE'] = data['kAperMag3'] - data['DISTANCE_MODULUS']

In [ ]:
ukidss_ordered_bands = ['Y', 'J', 'H', 'K']

fig, axes = plt.subplots(ncols=len(ukidss_ordered_bands), figsize=(20, 3))
for axes_n, band in enumerate(ukidss_ordered_bands):
    target_column = band + '_MAG_ABSOLUTE'
    axes[axes_n].hist(remove_bad_values(data[target_column]), bins=100)
    axes[axes_n].set_xlabel(target_column)
    axes[axes_n].set_xlim([-25, -14])
axes[0].set_ylabel('Galaxies')
fig.tight_layout()

#### UKIDSS Flux


In [ ]:
# UKIDSS is also on the AB magnitude system, so we can apply the same conversion as for SDSS
for band in ukidss_ordered_bands:
    data[band + '_FLUX_ABSOLUTE'] = ab_mag_to_flux(data[band + '_MAG_ABSOLUTE'])

In [ ]:
bins=50
pd.Series(np.log10(data['Y_FLUX_ABSOLUTE'])).hist(bins=bins, alpha=0.5, log=True)

In [ ]:
np.log10(data['Y_FLUX_ABSOLUTE']) < 4

In [ ]:
y_flux = pd.Series(data['Y_FLUX_ABSOLUTE'])
y_flux[np.log10(y_flux) < 4][:20]

In [ ]:
# would get 3316 if mag is 0
y_mag = pd.Series(data['Y_MAG_ABSOLUTE'])
y_mag[y_mag > -10]

#### Derived Mass (from NSA)

These are derived through the spectra without OSSY re-processing - presumably from the originals or from MPA-JHU.

**TODO: I should look up exactly where this comes from**

In [ ]:
log_mass = np.log(remove_bad_values(data['ELPETRO_MASS']))
log_mass = log_mass[(log_mass > 17) & (log_mass < 26)]  # remove crazy tails

plt.clf()
fig, axes = plt.subplots(ncols=1, figsize=(4, 3))
axes.hist(log_mass, bins=n_bins)
axes.set_xlabel('log mass (SDSS)')
# axes[0].set_ylabel('Galaxies')
fig.tight_layout()

#### Color-Magnitude Diagram

In [ ]:
# NSA columns with 7 values are for FNugriz values
g_mag = - np.array(data['ELPETRO_ABSMAG'])[:, 3]
r_mag = - np.array(data['ELPETRO_ABSMAG'])[:, 4]
g_r_color = pd.Series(g_mag - r_mag)

color_mag_df = pd.DataFrame(data={'g_r_color': g_r_color, 'g_mag': g_mag})
color_mag_df = color_mag_df.dropna(how='any')
color_mag_df = color_mag_df[(color_mag_df['g_mag'] > 16) & (color_mag_df['g_mag'] < 22)]
color_mag_df = color_mag_df[(color_mag_df['g_r_color'] > -1.) & (color_mag_df['g_r_color'] < 0.)]

In [ ]:
plt.hist2d(color_mag_df['g_r_color'], color_mag_df['g_mag'], bins=300)
plt.xlabel('G - R color')
plt.ylabel('G magnitude')
plt.title('Color-magnitude diagram')
plt.tight_layout()

#### OSSY Sanity Check: H-Alpha Lines

In [ ]:
h_alpha_raw = pd.Series(data['HA_6562'])
h_alpha = remove_bad_values(h_alpha_raw)
print('{} of {} galaxies have bad/missing h-alpha OSSY values'.format(len(h_alpha) - len(h_alpha_raw), len(h_alpha_raw)))
print('{} of {} galaxies have no h-alpha line'.format(np.sum(h_alpha == 0), len(h_alpha)))
_ = plt.hist(np.log(h_alpha[h_alpha > 0]), bins=50)
plt.xlabel('log OSSY flux (log $10^{-17} erg s^{-1} A^{-1}$)')
plt.ylabel('Galaxies')
plt.tight_layout()

## WISE

In [ ]:
wise_cols = ['AllWISE', 'RAJ2000', 'DEJ2000', 'eeMaj', 'eeMin', 'eePA', 'W1mag', 'W2mag', 'W3mag', 'W4mag', 'Jmag', 'Hmag', 'Kmag', 'e_W1mag', 'e_W2mag', 'e_W3mag', 'e_W4mag', 'e_Jmag', 'e_Hmag', 'e_Kmag', 'ID', 'ccf', 'ex', 'var', 'qph', 'pmRA', 'e_pmRA', 'pmDE', 'e_pmDE', 'd2M', 'angDist_wisex']

In [ ]:
wise_apparent_mag_bands = [
    'W1mag',
    'W2mag',
    'W3mag',
    'W4mag',
]

fig, axes = plt.subplots(ncols=len(wise_apparent_mag_bands), figsize=(20, 3))
for axes_n, mag in enumerate(wise_apparent_mag_bands):
    axes[axes_n].hist(remove_bad_values(data[mag]), bins=50)
    axes[axes_n].set_xlabel(mag)
axes[0].set_ylabel('Galaxies')
fig.tight_layout()

In [ ]:
wise_abs_mag_bands = [
    'W1_MAG_ABSOLUTE',
    'W2_MAG_ABSOLUTE',
    'W3_MAG_ABSOLUTE',
    'W4_MAG_ABSOLUTE'
]


# Note: these are not k-corrected
data['W1_MAG_ABSOLUTE'] = data['W1mag'] - data['DISTANCE_MODULUS']
data['W2_MAG_ABSOLUTE'] = data['W2mag'] - data['DISTANCE_MODULUS']
data['W3_MAG_ABSOLUTE'] = data['W3mag'] - data['DISTANCE_MODULUS']
data['W4_MAG_ABSOLUTE'] = data['W4mag'] - data['DISTANCE_MODULUS']

In [ ]:
fig, axes = plt.subplots(ncols=len(wise_abs_mag_bands), figsize=(20, 3))
for axes_n, mag in enumerate(wise_abs_mag_bands):
    axes[axes_n].hist(remove_bad_values(data[mag]), bins=100)
    axes[axes_n].set_xlabel(mag)
    axes[axes_n].set_xlim([-33, -15])
axes[0].set_ylabel('Galaxies')
fig.tight_layout()

There are some REALLY bright W4 sources, if my crossmatching and cosmology is correct! I'm a little skeptical of the cross-match, but unsure how to check.

#### Add WISE flux

For now, I won't do the WISE color correction for steep sources. It looks like a couple of hours work.

In [ ]:
for band in ['W1', 'W2', 'W3', 'W4']:
    data[band + '_FLUX_ABSOLUTE'] = shared_notebook_utils.wise_mag_to_flux(data[band + '_MAG_ABSOLUTE'], band)
    data[band + '_FLUX_ABSOLUTE'].unit = units.Jy  # update column units

In [ ]:
data[[band + '_FLUX_ABSOLUTE' for band in ['W1', 'W2', 'W3', 'W4']]][:5]

### SDSS Herschel Catalog

Ellison, S. L., Teimoorinia, H., Rosario, D. J., & Trevor Mendel, J. (2016). The infrared luminosities of ~332 000 SDSS galaxies predicted from artificial neural networks and the Herschel Stripe 82 survey. Monthly Notices of the Royal Astronomical Society, 455(1), 370–385. https://doi.org/10.1093/mnras/stv2275
    


> we repeat the training procedure 25 times and select the best 20 trained networks...The output of the best 20 trained networks show small differences due to the different initializations... for those galaxies in the SDSS sample that have very different [properties to Herschel-observed galaxies] the scatter will be very large...We adopt the mean $L_{IR}$ value of the 20 best trained networks, and assign the associated ‘error’ (σANN) as the scatter amongst the network outputs



In [ ]:
fir_loc = 'catalogs/ellison_inferred_fir/LIR_select.out'

# schema from Table 2
columns = ['sdss_objid', 'ra', 'dec', 'z', 'log_lir', 'std_dev']
fir = pd.read_csv(fir_loc, header=None, names=columns, sep=' ')


In [ ]:
print(len(fir))
fir.head()

In [ ]:
fir['z'].hist(bins=50)
plt.axvline(0.15, c='r', linestyle='--')
_ = plt.xlabel('z')
_ = plt.ylabel('Galaxies')

In [ ]:
# check there are no duplicates
assert not np.sum(fir.duplicated())
assert not np.sum(fir.duplicated(subset=['sdss_objid']))
assert not np.sum(fir.duplicated(subset=['ra', 'dec']))

#### Cross-match into the core sample, but keep rows with missing data

In [ ]:
data['ra'] = data['RA_1']
data['dec'] = data['DEC_1']

In [ ]:
fir_data = Table.from_pandas(fir)

In [ ]:
fir_data[:5]

In [ ]:
data[:5]

In [ ]:
test_columns = list(set(data.colnames) - set([x + '_FLUX_ABSOLUTE' for x in ukidss_ordered_bands]))

In [ ]:
matched, unmatched = matching_utils.match_galaxies_to_catalog_table(
    fir_data, 
    data[data.colnames], 
    join_type='right',  # keep catalog rows even if they have no galaxy match
    matching_radius = 5 * units.arcsec, 
    galaxy_suffix='_fir')

In [ ]:
assert len(matched) == len(data)

In [ ]:
print(len(data))
print(len(matched))

In [ ]:
sdss

In [ ]:
def check_id_fields_are_unique(data):
    for id_col in ['sdss_objid', 'sourceID', 'SDSS_ID', 'IAUNAME']:
        counts = Counter(list(filter(lambda x: x > 0, matched[id_col])))  # count all obj ids
        try:
            assert counts.most_common(1)[0][1] == 1  # most common id occurs only once
        except AssertionError:
            print(id_col)
            print(counts.most_common(10))
            return counts

In [ ]:
bad_counts = check_id_fields_are_unique(data)

In [ ]:
duplicate_source_id_tuples = filter(lambda x: x[1] > 1, counts.most_common(50))
duplicate_source_ids = [pair[0] for pair in duplicate_source_id_tuples]

In [ ]:
duplicate_rows = data[data['sourceID'] == duplicate_source_ids[4]]
duplicate_rows

In [ ]:
check_id_fields_are_unique(matched)

### Save any newly-derived data

In [ ]:
# matched.write(data_loc.rstrip('.fits') + '_derived.fits', overwrite=True)

In [ ]:
assert False   # stop execution here

### Deprecated

#### Extract and convert to Janksy


From [here](http://www.sdss3.org/dr8/algorithms/magnitudes.php#nmgy): 

> A "maggy" is the flux f of the source relative to the standard source f0 (which defines the zeropoint of the magnitude scale). Therefore, a "nanomaggy" is $10^{-9}$ times a maggy. 

> The standard source for each SDSS band is close to but not exactly the AB source (3631 Jy), meaning that a nanomaggy is approximately 3.631×10-6 Jy

Let's use cosmology distance estimates to convert to luminosity. See [here](https://ned.ipac.caltech.edu/level5/Hogg/Hogg7.html)

In [ ]:
data['luminosity_distance'] = cosmo.luminosity_distance(data['Z'])  

In [ ]:
pd.Series(data['luminosity_distance']).hist()

Luminosity distance is distance as far as solid angle subtended is concerned. See [here](https://en.wikipedia.org/wiki/Distance_measures_(cosmology))

In [ ]:
from astropy.cosmology import z_at_value

In [ ]:
# reference_z = z_at_value(cosmo.luminosity_distance, 10 * units.pc)  # z of 10 pc
# reference_lum_distance = cosmo.luminosity_distance(reference_z)  
# reference_lum_distance  # okay, it's essentially the same...but at least I know how to do this now

reference_lum_distance = 10 * units.pc
for band in ordered_sdss_bands:
    data[band + '_FLUX_ABSOLUTE'] = np.power(data['luminosity_distance'] / 10 * reference_lum_distance, 2) * data[band + '_FLUX_APPARENT']


From [here](http://www.sdss3.org/dr8/algorithms/magnitudes.php#nmgy): 

> Magnitudes within the SDSS are expressed as inverse hyperbolic sine (or "asinh") magnitudes, described in detail by Lupton, Gunn & Szalay (1999). They are sometimes referred to informally as luptitudes. The transformation from linear flux measurements to asinh magnitudes is designed to be virtually identical to the standard astronomical magnitude at high signal-to-noise ratio, but to behave reasonably at low signal-to-noise ratio

$$ m = \frac{-2.5}{\ln(10)} * [asinh(\frac{f/f_0}{2b}) + \ln(b)] $$



From [here](https://www.sdss.org/dr13/algorithms/fluxcal/#counts2mag): 
    
> Assuming you know the correction from SDSS zeropoints to AB zeropoints (see above), you can turn the AB magnitudes into a flux density using the AB zeropoint flux density. The AB system is defined such that every filter has a zero-point flux density of 3631 Jy (1 Jy = 1 Jansky = 10-26 W Hz-1 m-2 = 10-23 erg s-1 Hz-1 cm-2).
    
> To obtain a flux density from SDSS data, you need to work out f/f0 (e.g. from the asinh magnitudes in the photoObj files by using the inverse of the relations given on the magnitudes page). 

I think this is $$ \frac{f}{f_0} = asinh^{-1} (-m \frac{ln(10)}{2.5} - ln(b)) * 2b$$

In [ ]:
sdss_flux_calibration = pd.DataFrame(data=[
    {
        'filter': 'u',
        'b': 1.4e-10,
        'zero_magnitude': 24.63
    },
    
    {
        'filter': 'g',
        'b': 0.9e-10,
        'zero_magnitude': 25.11
    },
    
    {
        'filter': 'r',
        'b': 1.2e-10,
        'zero_magnitude': 24.8
    },
    
    {
        'filter': 'i',
        'b': 1.8e-10,
        'zero_magnitude': 24.36
    },
    
    {
        'filter': 'z',
        'b': 7.4e-10,
        'zero_magnitude': 22.83
    },
    
]).set_index('filter')
# see http://horus.roe.ac.uk:8080/UKIDSSphotometry/magnitudes.html
sdss_flux_calibration['f0'] = 10 ** (0.4 * sdss_flux_calibration['zero_magnitude'] )

In [ ]:
sdss_flux_calibration

In [ ]:
def f_over_f0(m, b):
    arcsinh_arg = (-m * np.log(10) / 2.5) - np.log(b)
    print(arcsinh_arg)
    return np.arcsinh(arcsinh_arg) * 2 * b  # dimensionless. b is tiny, hence output is tiny

>This number is then the also the object’s flux density, expressed as fraction of the AB zeropoint flux density. 

> The AB system is defined such that every filter has a zero-point flux density of 3631 Jy 

> Therefore, the conversion to flux density is

$$S[Jy] = 3631 * \frac{f}{f_0} $$

In [ ]:
def S(f_ratio):  # slightly weirdly, this does not depend on f0
    return 3631 * f_ratio

> Then you need to apply the correction for the zeropoint offset between the SDSS system and the AB system. See the description of SDSS to AB conversion above.

The 'above' part is actually describing a *magnitude* correction, not the zero-point offset. Let's leave this for now - it's never more than 0.04 mag.

In [ ]:
f_f0 = f_over_f0(data['U_MAG_ABSOLUTE'][:10], sdss_flux_calibration.loc['u']['b'])
print(f_f0)
data['U_FLUX_ABSOLUTE_DIRECT'][:10] = S(f_f0)

In [ ]:
fig, axes = plt.subplots(2)
bins=50
pd.Series(np.log10(data['U_FLUX_ABSOLUTE'])).hist(ax=axes[0], bins=bins, alpha=0.5, log=True)
pd.Series(np.log10(data['U_FLUX_ABSOLUTE_DIRECT'])).hist(ax=axes[1], bins=bins, alpha=0.5, log=True)


See [here](http://horus.roe.ac.uk:8080/UKIDSSphotometry/magnitudes.html). UKIDSS uses the SDSS system, but with an added twist for b.

$$ b = \beta * \frac{\sigma_{sky}}{f_0} $$ 

> The default value β = 1.042

In [ ]:
beta = 1.042  # approx. 1

In [ ]:
las_sky_noise_f0_ratio = {
    'Y': 3e-10,
    'J': 0.9e-10,
    'H': 5.2e-10,
    'K': 1.1e-9,
}  

In [ ]:
band = 'Y' # for now

b = beta * las_sky_noise_f0_ratio[band]  # b will be fairly similar to the SDSS band b-values
data['Y_FLUX_ABSOLUTE_DIRECT'] = S(f_over_f0(data['Y_MAG_ABSOLUTE'], b))

In [ ]:
pd.Series(np.log10(data['Y_FLUX_ABSOLUTE_DIRECT'])).hist(bins=bins, log=True)

In [ ]:

reference_lum_distance = 10 * units.pc
for band in ordered_sdss_bands:
    data[band + '_FLUX_ABSOLUTE'] = np.power(data['luminosity_distance'] / 10 * reference_lum_distance, 2) * data[band + '_FLUX_APPARENT']